In [94]:
import json
import os
from tqdm import tqdm
import pandas as pd
import re
import numpy as np
import datetime
import dateutil.parser as dp
from dateutil.relativedelta import relativedelta

In [95]:
from nltk.corpus import stopwords
from nltk import FreqDist
import collections
import string

def clean_post(post_text):
    translator = str.maketrans('', '', string.punctuation) # for removing punctuation
    post_text = post_text.translate(translator)
    token_text = [t for t in post_text.replace('\n',' ').split()]
    
    sr = stopwords.words('english')
    sr.extend(['gay', 'bear', 'girlfriend', 'fuckers', 'bull'])
    cleanTokens = [i for i in token_text if i not in sr]
    
    return cleanTokens

nasdaq = pd.read_csv('tickers/nasdaq.csv')
nyse = pd.read_csv('tickers/nyse.csv')
#nasdaq['Symbol'] = nasdaq['Symbol'].apply(lambda x: x.lower())
#nyse['Symbol'] = nyse['Symbol'].apply(lambda x: x.lower())

def check_tickers(tokens):
    ticker_counter = collections.Counter()
    for token in tokens:
        if any(nasdaq['Symbol'].isin([token])): 
            ticker_counter[token] += 1
            print(token)
        if any(nyse['Symbol'].isin([token])): 
            ticker_counter[token] += 1
            print(token)
    return ticker_counter

In [96]:
def ticker_finder(row):
    if row['regexed_combined'] == 0:
        return
    if type(row['selftext']) == float:
        return
    lookup_string = row['title'] + ' ' + row['selftext']
    #lookup_string = lookup_string.lower()
    tickers = check_tickers(clean_post(lookup_string))
    return tickers

In [97]:
def ticker_adder(pos_list):
    prev_ticker = "NONE"
    for group in pos_list:
        if not group[0]:
            group[0] = prev_ticker
        else:
            prev_ticker = group[0]
    return pos_list

In [143]:
def regex_pos(post_text):
    if pd.isna(post_text): return 0
    x = []
    p1 = re.compile('(\$?[A-Z]{1,4})?\s+\$?(?:(\d+(?:\.\d\d?)?)\s*([CcPp])\w{0,4}\s+(\d\d?\/\d\d?(?:\/\d{2,4})?))')
    p2 = re.compile('(\$?[A-Z]{1,4})?\s+(\d\d?\/\d\d?(?:\/\d{2,4})?)\s+\$?(\d+(?:\.\d\d?)?)\s*([CcPp])\w{0,4}')
    p3 = re.compile('(\d\d?\/\d\d?(?:\/\d{2,4})?)\s+(\$?[A-Z]{1,4})\s+\$?(\d+(?:\.\d\d?)?)\s*([CcPp])\w{0,4}')
    p4 = re.compile('(\$?[A-Z]{1,4})?\s+(\d{1,2}\s?[A-Z]{1,3})\s+\$?(\d+(?:\.\d\d?)?)\s*([CcPp])\w{0,4}')
    p5 = re.compile('(\d{1,2}\s?[A-Z]{1,3})\s+(\$?[A-Z]{1,4})?\s+\$?(\d+(?:\.\d\d?)?)\s*([CcPp])\w{0,4}')
    p6 = re.compile('([A-Z]{1,3}\s?\d{1,2})\s+(\$?[A-Z]{1,4})?\s+\$?(\d+(?:\.\d\d?)?)\s*([CcPp])\w{0,4}')
    p7 = re.compile('(\$?[A-Z]{1,4})?\s+([A-Z]{1,3}\s?\d{1,2})\s+\$?(\d+(?:\.\d\d?)?)\s*([CcPp])\w{0,4}')
    
    # (\$?[A-Z]{1,4})\s+(?:\$?(\d+(?:.\d\d)?)\s*([CcPp])\w{0,4}\s+(\d\d?\/\d\d?(?:\/\d{2,4})?)+[^A-Z]*)
    
    x1 = re.findall(p1, post_text)
    x2 = re.findall(p2, post_text)
    x3 = re.findall(p3, post_text)
    x4 = re.findall(p4, post_text) # diving into alpha-numeric dates like 20 NOV
    x5 = re.findall(p5, post_text) #alpha 20 nov, date first
    x6 = re.findall(p6, post_text) #alpha nov 20, date first
    x7 = re.findall(p7, post_text) #alpha nov 20, ticker first
    
    #dictating ticker-strike-c/p-date as standard. x1 stays, the rest must be edited 
    if x1:
        x1_e = []
        for match in x1:
            ordered = [match[0], match[1], match[2], match[3]]
            x1_e.append(ordered)
        x.extend(x1_e)
    if x2: #ticker-date-strike-c/p
        x2_e = []
        for match in x2:
            ordered = [match[0],match[2],match[3],match[1]]
            x2_e.append(ordered)
        x.extend(x2_e)
    if x3: #date-ticker-strike-c/p
        x3_e = []
        for match in x3:
            ordered = [match[1],match[2],match[3],match[0]]
            x3_e.append(ordered)
        x.extend(x3_e)
    if x4: # ticker-date(alphanumeric)-stike-c/p
        x4_e = []
        for match in x4:
            ordered = [match[0],match[2],match[3],match[1]]
            x4_e.append(ordered)
        x.extend(x4_e)
    if x5: #date(alpha)-ticker-strike-cp
        x5_e = []
        for match in x5:
            ordered = [match[1],match[2],match[3],match[0]]
            x5_e.append(ordered)
        x.extend(x5_e)
    if x6: #same as above, diff alpha
        x6_e = []
        for match in x6:
            ordered = [match[1],match[2],match[3],match[0]]
            x6_e.append(ordered)
        x.extend(x6_e)
    if x7:
        x7_e = []
        for match in x7:
            ordered = [match[0],match[2],match[3],match[1]]
            x7_e.append(ordered)
        x.extend(x7_e)

    if x:
        x = ticker_adder(x)
        return x 
    else:
        return 0

In [ ]:
def title_body_combiner(row):
    if row['regexed_body'] == 0: # we either return a zero b/c both are, or we get info. could be vice versa and still work
        return row['regexed_title']
    # now we have to compare each of body and title. if its all the same, there is nothing to do
    # if we find a situation where one has the ticker and the other doesn't, use the ticker ONLY if all other info is 
    # the same. 
    if row['regexed_body'] == row['regexed_title']: 
        return row['regexed_body']
    # now the hard for loops


In [177]:
# adds -1 to unproccessed dates, these are worth dropping
def date_proccessor(pos_list):
    if pos_list == 0: return pos_list
    new_list = pos_list
    for position in new_list:
        date = position[-1]
        try:
            p_date = dp.parse(date)
        except:
            try:
                date_e = date.split('/')
                if len(date_e) == 2:
                    date_e = '/'.join(i for i in date_e[::-1])
                elif len(date_e) == 3:
                    tmp = date_e[0]
                    date_e[0] = date_e[1]
                    date_e[1] = tmp
                    date_e = '/'.join(i for i in date_e[::-1])
                p_date = dp.parse(date_e)                
            except:
                print("date parse error, probably worth dropping")
                print(date)
                p_date = -1
        position[-1] = p_date
    return new_list

def date_proccessor_corrector(pos_list):
    if pos_list == 0: return pos_list
    new_list = []
    for position in pos_list:
        if position[-1] != -1:
            new_list.append(position)
    if new_list:
        return new_list
    else:
        return 0

In [197]:
def expiry_year_corrector(row):
    post_date = int(row['created_utc'])
    post_date = datetime.datetime.fromtimestamp(post_date)
    combined = row['regexed_combined']
    if combined == 0: return combined
    current_year = datetime.datetime.now().year
    for position in combined:
        expiry = position[3]
        expiry_year = expiry.year
        expiry_month = expiry.month
        post_year = post_date.year
        post_month = post_date.month
        
        if expiry_year > current_year: 
            return combined # no changes needed
        
        if expiry_month >= post_month: # expiry month is greater than month, is POST year
            expiry = datetime.datetime(post_year,expiry.month,expiry.day)
        elif expiry_month < post_month: # expiry month is less than post month, is POST year + 1
            expiry = datetime.datetime(post_year+1,expiry.month,expiry.day)
        position[3] = expiry
    print(combined)
    return combined
        
    # get the created year, compare to expiry
    # make adjustments as needed
    # return

In [101]:
## maybe create a number of posts function for a new col
## need a expiry function to get expiry out of position
## stock at open and close (where close applicable)

In [213]:
good_post_csv = pd.read_csv('all_good_posts.csv')
all_post_csv = pd.read_csv('all_posts.csv')

In [214]:
good_post_csv['all text'] = good_post_csv['title'] + good_post_csv['selftext']

In [215]:
good_post_csv['regexed_combined'] = good_post_csv['all text'].apply(regex_pos)
good_post_csv['regexed_body'] = good_post_csv['selftext'].apply(regex_pos)
good_post_csv['regexed_title'] = good_post_csv['title'].apply(regex_pos)

In [217]:
good_post_csv[good_post_csv['regexed_title'] != 0]

,Unnamed: 0,author,created_utc,title,link_flair_text,selftext,id,full_link,all text,regexed_combined,regexed_body,regexed_title
136,136,clmohn,1585011346,COVID-19 was just something printing me tendie...,Discussion,[removed],fnzzcm,https://www.reddit.com/r/wallstreetbets/commen...,COVID-19 was just something printing me tendie...,"[[NONE, 200, P, 5/15]]",0,"[[NONE, 200, P, 5/15]]"
306,306,B1G_Peter,1585043577,Universities are starting to bring back studen...,Discussion,[https://www.newsadvance.com/news/local/thousa...,fo6e2g,https://www.reddit.com/r/wallstreetbets/commen...,Universities are starting to bring back studen...,"[[SPY, 50, p, 4/17], [SPY, 222, p, 3/30]]","[[SPY, 222, p, 3/30]]","[[SPY, 50, p, 4/17]]"
528,528,llllllllhhhhhhhhh,1585059667,Walmart 120c 4/17,Discussion,So I got this bad boy this morning. I don’t an...,fob8vs,https://www.reddit.com/r/wallstreetbets/commen...,Walmart 120c 4/17So I got this bad boy this mo...,"[[NONE, 120, c, 4/17]]",0,"[[NONE, 120, c, 4/17]]"
743,743,cleveland-216,1585073810,GE 7c 3/27,Options,Going all in on GE futures at 7c 3/27 on a qui...,fofhbq,https://www.reddit.com/r/wallstreetbets/commen...,GE 7c 3/27Going all in on GE futures at 7c 3/2...,"[[GE, 7, c, 3/27], [GE, 7, c, 3/27]]","[[NONE, 7, c, 3/27]]","[[GE, 7, c, 3/27]]"
750,750,futuretrollshark,1585074464,Buy MTCH 60c 4/17,DD,Tinder had a 15% UPTICK in downloads. Wanna k...,fofnzc,https://www.reddit.com/r/wallstreetbets/commen...,Buy MTCH 60c 4/17Tinder had a 15% UPTICK in do...,"[[MTCH, 60, c, 4/17], [MTCH, 60, c, 4/17]]","[[NONE, 60, c, 4/17]]","[[MTCH, 60, c, 4/17]]"
...,...,...,...,...,...,...,...,...,...,...,...,...
117438,117438,maR-08,1604926020,ICLN $26C 1/15/20 To the MOON 🚀🚀,Discussion,NaN,jr2c1x,https://www.reddit.com/r/wallstreetbets/commen...,NaN,0,0,"[[ICLN, 26, C, 1/15/20]]"
117439,117439,maR-08,1604926092,ICLN $26 Call 1/15/21 🔥🔥🚀🚀,Discussion,NaN,jr2cvt,https://www.reddit.com/r/wallstreetbets/commen...,NaN,0,0,"[[ICLN, 26, C, 1/15/21]]"
117467,117467,Dan_inKuwait,1604928694,BNTX 105c 11/20 - diamond hands or eat the mar...,Discussion,"Hey, king retard here.\n\n&amp;#x200B;\n\nIn a...",jr386y,https://www.reddit.com/r/wallstreetbets/commen...,BNTX 105c 11/20 - diamond hands or eat the mar...,"[[BNTX, 105, c, 11/20], [$BNTX, 105, c, 11/20]]","[[$BNTX, 105, c, 11/20]]","[[BNTX, 105, c, 11/20]]"
117557,117557,TheGuyWhoCodes,1604938238,NIO 43c 11/13 How we feeling?,Discussion,NaN,jr6gtz,https://www.reddit.com/r/wallstreetbets/commen...,NaN,0,0,"[[NIO, 43, c, 11/13]]"


In [224]:
#good_post_csv.to_csv('output1.csv')

In [222]:
good_post_csv.iloc[743]['title']

'GE 7c 3/27'

In [219]:
good_post_csv.iloc[743]['selftext']

'Going all in on GE futures at 7c 3/27 on a quick play based off of announcement of the collaboration with 3M and ford to make masks and ventilators. Up 133% today and seems promising for a quick flip and potentially looking at a longer play. Any thoughts?'

In [223]:
good_post_csv.iloc[743]['all text']

'GE 7c 3/27Going all in on GE futures at 7c 3/27 on a quick play based off of announcement of the collaboration with 3M and ford to make masks and ventilators. Up 133% today and seems promising for a quick flip and potentially looking at a longer play. Any thoughts?'

In [220]:
good_post_csv.iloc[743]['regexed_title']

[['GE', '7', 'c', '3/27']]

In [205]:
# add title to empty body in new col
#good_post_csv['regexed_combined'] = np.where(good_post_csv['regexed_body'] == 0, good_post_csv['regexed_title'], good_post_csv['regexed_body'])

In [206]:
good_post_csv[good_post_csv['regexed_combined'] != 0]

,Unnamed: 0,author,created_utc,title,link_flair_text,selftext,id,full_link,regexed_body,regexed_title,regexed_combined
6,6,angryrantingdude,1569872663,Should I cut my loss or turn my puts into a st...,Options,So I bought $T $37.5p 10/4 expiration. But ob...,dblal0,https://www.reddit.com/r/wallstreetbets/commen...,"[[$T, 37.5, p, 10/4]]",0,"[[$T, 37.5, p, 10/4]]"
136,136,clmohn,1585011346,COVID-19 was just something printing me tendie...,Discussion,[removed],fnzzcm,https://www.reddit.com/r/wallstreetbets/commen...,0,"[[NONE, 200, P, 5/15]]","[[NONE, 200, P, 5/15]]"
159,159,lost_civilizations,1585016402,Hotels and Airline bailouts,Fundamentals,Wake up people - The only reason these compani...,fo0vym,https://www.reddit.com/r/wallstreetbets/commen...,"[[HLT, 40, P, 5/1]]",0,"[[HLT, 40, P, 5/1]]"
161,161,CodfishCannon,1585017606,AMZN Call Spread Bug,YOLO,Found a fun little Robinhood glitch: more valu...,fo13kb,https://www.reddit.com/r/wallstreetbets/commen...,"[[HDB, 35, p, 4/17], [EWP, 19, p, 4/17], [BLK,...",0,"[[HDB, 35, p, 4/17], [EWP, 19, p, 4/17], [BLK,..."
210,210,jlk1994-,1585034103,Was I misdiagnosed?,Discussion,"I got diagnosed with Asperger, like an actual ...",fo44sm,https://www.reddit.com/r/wallstreetbets/commen...,"[[VTI, 70, p, 4/17]]",0,"[[VTI, 70, p, 4/17]]"
...,...,...,...,...,...,...,...,...,...,...,...
117549,117549,yeeterboi1234567,1604937591,Who else went all in on RKT today?,YOLO,Went all in on RKT 11/13 25c lol. Who else?,jr68pb,https://www.reddit.com/r/wallstreetbets/commen...,"[[RKT, 25, c, 11/13]]",0,"[[RKT, 25, c, 11/13]]"
117557,117557,TheGuyWhoCodes,1604938238,NIO 43c 11/13 How we feeling?,Discussion,NaN,jr6gtz,https://www.reddit.com/r/wallstreetbets/commen...,0,"[[NIO, 43, c, 11/13]]","[[NIO, 43, c, 11/13]]"
117570,117570,Babyyodaenergy,1604939196,Nio Calls DEC,Discussion,How are we feeling about December NIO calls \...,jr6slg,https://www.reddit.com/r/wallstreetbets/commen...,"[[NONE, 42, C, 12/11], [NONE, 50, C, 1/15/21]]",0,"[[NONE, 42, C, 12/11], [NONE, 50, C, 1/15/21]]"
117573,117573,gravitiz3,1604939411,PLTR gang,Discussion,Y’all holding long term? Where do you see it h...,jr6v8s,https://www.reddit.com/r/wallstreetbets/commen...,"[[NONE, 10, c, 5/21/20]]",0,"[[NONE, 10, c, 5/21/20]]"


In [207]:
good_post_csv['regexed_combined'] = good_post_csv['regexed_combined'].apply(date_proccessor)

date parse error, probably worth dropping
SPY 25
date parse error, probably worth dropping
SPY 15
date parse error, probably worth dropping
Q3
date parse error, probably worth dropping
20 SPY
date parse error, probably worth dropping
35 SPY
date parse error, probably worth dropping
50P
date parse error, probably worth dropping
LEM20
date parse error, probably worth dropping
GFK20
date parse error, probably worth dropping
HEK20
date parse error, probably worth dropping
DLK20
date parse error, probably worth dropping
1 OTM
date parse error, probably worth dropping
30 SPY
date parse error, probably worth dropping
10 USD
date parse error, probably worth dropping
5
SLV
date parse error, probably worth dropping
10
UPS
date parse error, probably worth dropping
0 DTE
date parse error, probably worth dropping
0 DTE
date parse error, probably worth dropping
0 DTE
date parse error, probably worth dropping
0 DTE
date parse error, probably worth dropping
2/30
date parse error, probably worth droppi

In [208]:
x[136]

[['NONE', '200', 'P', datetime.datetime(2020, 5, 15, 0, 0)]]

In [209]:
good_post_csv['regexed_combined'] = good_post_csv['regexed_combined'].apply(date_proccessor_corrector)

In [210]:
good_post_csv[good_post_csv['regexed_combined'] != 0]

,Unnamed: 0,author,created_utc,title,link_flair_text,selftext,id,full_link,regexed_body,regexed_title,regexed_combined
6,6,angryrantingdude,1569872663,Should I cut my loss or turn my puts into a st...,Options,So I bought $T $37.5p 10/4 expiration. But ob...,dblal0,https://www.reddit.com/r/wallstreetbets/commen...,"[[$T, 37.5, p, 2020-10-04 00:00:00]]",0,"[[$T, 37.5, p, 2020-10-04 00:00:00]]"
136,136,clmohn,1585011346,COVID-19 was just something printing me tendie...,Discussion,[removed],fnzzcm,https://www.reddit.com/r/wallstreetbets/commen...,0,"[[NONE, 200, P, 2020-05-15 00:00:00]]","[[NONE, 200, P, 2020-05-15 00:00:00]]"
159,159,lost_civilizations,1585016402,Hotels and Airline bailouts,Fundamentals,Wake up people - The only reason these compani...,fo0vym,https://www.reddit.com/r/wallstreetbets/commen...,"[[HLT, 40, P, 2020-05-01 00:00:00]]",0,"[[HLT, 40, P, 2020-05-01 00:00:00]]"
161,161,CodfishCannon,1585017606,AMZN Call Spread Bug,YOLO,Found a fun little Robinhood glitch: more valu...,fo13kb,https://www.reddit.com/r/wallstreetbets/commen...,"[[HDB, 35, p, 2020-04-17 00:00:00], [EWP, 19, ...",0,"[[HDB, 35, p, 2020-04-17 00:00:00], [EWP, 19, ..."
210,210,jlk1994-,1585034103,Was I misdiagnosed?,Discussion,"I got diagnosed with Asperger, like an actual ...",fo44sm,https://www.reddit.com/r/wallstreetbets/commen...,"[[VTI, 70, p, 2020-04-17 00:00:00]]",0,"[[VTI, 70, p, 2020-04-17 00:00:00]]"
...,...,...,...,...,...,...,...,...,...,...,...
117549,117549,yeeterboi1234567,1604937591,Who else went all in on RKT today?,YOLO,Went all in on RKT 11/13 25c lol. Who else?,jr68pb,https://www.reddit.com/r/wallstreetbets/commen...,"[[RKT, 25, c, 2020-11-13 00:00:00]]",0,"[[RKT, 25, c, 2020-11-13 00:00:00]]"
117557,117557,TheGuyWhoCodes,1604938238,NIO 43c 11/13 How we feeling?,Discussion,NaN,jr6gtz,https://www.reddit.com/r/wallstreetbets/commen...,0,"[[NIO, 43, c, 2020-11-13 00:00:00]]","[[NIO, 43, c, 2020-11-13 00:00:00]]"
117570,117570,Babyyodaenergy,1604939196,Nio Calls DEC,Discussion,How are we feeling about December NIO calls \...,jr6slg,https://www.reddit.com/r/wallstreetbets/commen...,"[[NONE, 42, C, 2020-12-11 00:00:00], [NONE, 50...",0,"[[NONE, 42, C, 2020-12-11 00:00:00], [NONE, 50..."
117573,117573,gravitiz3,1604939411,PLTR gang,Discussion,Y’all holding long term? Where do you see it h...,jr6v8s,https://www.reddit.com/r/wallstreetbets/commen...,"[[NONE, 10, c, 2020-05-21 00:00:00]]",0,"[[NONE, 10, c, 2020-05-21 00:00:00]]"


In [211]:
good_post_csv['regexed_combined'] = good_post_csv.apply(expiry_year_corrector, axis=1)

[['$T', '37.5', 'p', datetime.datetime(2019, 10, 4, 0, 0)]]
[['NONE', '200', 'P', datetime.datetime(2020, 5, 15, 0, 0)]]
[['HLT', '40', 'P', datetime.datetime(2020, 5, 1, 0, 0)]]
[['HDB', '35', 'p', datetime.datetime(2020, 4, 17, 0, 0)], ['EWP', '19', 'p', datetime.datetime(2020, 4, 17, 0, 0)], ['BLK', '295', 'p', datetime.datetime(2020, 4, 17, 0, 0)], ['EEM', '19.73', 'p', datetime.datetime(2020, 9, 18, 0, 0)]]
[['VTI', '70', 'p', datetime.datetime(2020, 4, 17, 0, 0)]]
[['SPY', '200', 'p', datetime.datetime(2020, 4, 13, 0, 0)], ['SPY', '205', 'p', datetime.datetime(2020, 4, 15, 0, 0)], ['SPY', '200', 'p', datetime.datetime(2020, 5, 1, 0, 0)]]
[['V', '167.5', 'c', datetime.datetime(2020, 3, 27, 0, 0)]]
[['NONE', '9', 'c', datetime.datetime(2020, 5, 15, 0, 0)], ['NONE', '8.5', 'c', datetime.datetime(2020, 4, 17, 0, 0)]]
[['$SPCE', '13.5', 'p', datetime.datetime(2020, 3, 27, 0, 0)]]
[['CCL', '13', 'p', datetime.datetime(2020, 3, 27, 0, 0)], ['SEAS', '9', 'p', datetime.datetime(2020, 3, 2

[['JNPR', '23.50', 'C', datetime.datetime(2020, 2, 21, 0, 0)], ['MSFT', '190', 'C', datetime.datetime(2020, 2, 14, 0, 0)], ['AMD', '50.50', 'C', datetime.datetime(2020, 2, 14, 0, 0)]]
[['$CHGG', '60', 'c', datetime.datetime(2020, 3, 20, 0, 0)]]
[['SPY', '325', 'P', datetime.datetime(2020, 6, 30, 0, 0)]]
[['NONE', '150', 'C', datetime.datetime(2020, 4, 17, 0, 0)], ['NONE', '160', 'C', datetime.datetime(2020, 4, 17, 0, 0)]]
[['MSFT', '190', 'c', datetime.datetime(2020, 2, 14, 0, 0)]]
[['NONE', '200', 'c', datetime.datetime(2020, 3, 20, 0, 0)]]
[['MSFT', '200', 'c', datetime.datetime(2020, 3, 20, 0, 0)]]
[['CVS', '75', 'C', datetime.datetime(2020, 2, 14, 0, 0)]]
[['MSFT', '205', 'c', datetime.datetime(2020, 2, 21, 0, 0)], ['SPCE', '24', 'c', datetime.datetime(2020, 2, 21, 0, 0)]]
[['NONE', '230', 'c', datetime.datetime(2020, 12, 18, 0, 0)]]
[['MSFT', '200', 'c', datetime.datetime(2020, 2, 21, 0, 0)], ['MSFT', '200', 'c', datetime.datetime(2020, 3, 6, 0, 0)], ['SPCE', '29', 'c', datetime.d

[['$STNG', '25', 'c', datetime.datetime(2020, 7, 17, 0, 0)]]
[['NONE', '195', 'p', datetime.datetime(2020, 4, 24, 0, 0)], ['NONE', '175', 'p', datetime.datetime(2020, 5, 15, 0, 0)], ['NONE', '210', 'p', datetime.datetime(2020, 5, 15, 0, 0)]]
[['VXX', '55', 'c', datetime.datetime(2020, 3, 27, 0, 0)]]
[['TSLA', '275', 'P', datetime.datetime(2020, 3, 27, 0, 0)], ['AAPL', '200', 'P', datetime.datetime(2020, 3, 27, 0, 0)]]
[['SPXS', '25', 'c', datetime.datetime(2020, 3, 20, 0, 0)]]
[['HSIC', '50', 'p', datetime.datetime(2020, 4, 17, 0, 0)]]
[['$SPXS', '15', 'P', datetime.datetime(2020, 3, 20, 0, 0)]]
[['$MAR', '60', 'p', datetime.datetime(2020, 7, 17, 0, 0)]]
[['NONE', '15', 'p', datetime.datetime(2020, 5, 19, 0, 0)]]
[['SPY', '220', 'p', datetime.datetime(2020, 4, 1, 0, 0)]]
[['$AAL', '10', 'p', datetime.datetime(2020, 4, 9, 0, 0)]]
[['NONE', '245', 'p', datetime.datetime(2020, 4, 17, 0, 0)]]
[['$VKTX', '4', 'p', datetime.datetime(2020, 3, 20, 0, 0)], ['$FIT', '6.5', 'p', datetime.datetime

[['NCLH', '12', 'p', datetime.datetime(2020, 4, 3, 0, 0)]]
[['NCLH', '12', 'p', datetime.datetime(2020, 4, 3, 0, 0)]]
[['CCRN', '10', 'c', datetime.datetime(2020, 6, 19, 0, 0)], ['COP', '34', 'c', datetime.datetime(2020, 4, 9, 0, 0)], ['CZR', '7', 'c', datetime.datetime(2020, 4, 3, 0, 0)], ['MAR', '100', 'c', datetime.datetime(2020, 4, 3, 0, 0)], ['MSFT', '200', 'c', datetime.datetime(2020, 3, 20, 0, 0)], ['MSFT', '200', 'c', datetime.datetime(2020, 3, 20, 0, 0)], ['SPY', '300', 'c', datetime.datetime(2020, 4, 17, 0, 0)], ['SPY', '300', 'c', datetime.datetime(2020, 5, 1, 0, 0)], ['SPY', '300', 'c', datetime.datetime(2020, 5, 15, 0, 0)], ['SPY', '350', 'c', datetime.datetime(2020, 4, 17, 0, 0)], ['SPY', '369', 'c', datetime.datetime(2020, 4, 20, 0, 0)], ['SPY', '400', 'c', datetime.datetime(2020, 4, 3, 0, 0)], ['SPY', '420', 'c', datetime.datetime(2020, 4, 1, 0, 0)], ['SPY', '420', 'c', datetime.datetime(2020, 4, 20, 0, 0)], ['SPY', '420', 'c', datetime.datetime(2020, 6, 9, 0, 0)], ['SP

[['SPY', '295', 'c', datetime.datetime(2020, 5, 15, 0, 0)], ['SPY', '295', 'c', datetime.datetime(2020, 5, 15, 0, 0)]]
[['SPY', '300', 'C', datetime.datetime(2020, 5, 15, 0, 0)]]
[['SPY', '300', 'C', datetime.datetime(2020, 5, 15, 0, 0)]]
[['$LOGI', '50', 'p', datetime.datetime(2020, 5, 15, 0, 0)], ['$LOGI', '45', 'p', datetime.datetime(2020, 6, 19, 0, 0)], ['$LOGI', '45', 'p', datetime.datetime(2020, 5, 15, 0, 0)]]
[['SPY', '255', 'p', datetime.datetime(2020, 7, 17, 0, 0)]]
[['NONE', '295', 'c', datetime.datetime(2020, 5, 18, 0, 0)]]
[['NONE', '30', 'c', datetime.datetime(2020, 11, 20, 0, 0)]]
[['SPY', '280', 'p', datetime.datetime(2020, 8, 21, 0, 0)]]
[['NONE', '210', 'c', datetime.datetime(2020, 5, 15, 0, 0)]]
[['SPY', '280', 'p', datetime.datetime(2020, 8, 21, 0, 0)]]
[['NONE', '55', 'C', datetime.datetime(2020, 6, 19, 0, 0)]]
[['NONE', '6', 'c', datetime.datetime(2020, 9, 18, 0, 0)], ['NONE', '6', 'c', datetime.datetime(2020, 7, 17, 0, 0)]]
[['DHT', '7', 'c', datetime.datetime(202

[['SPY', '280', 'p', datetime.datetime(2020, 6, 19, 0, 0)]]
[['NONE', '50', 'P', datetime.datetime(2020, 6, 26, 0, 0)]]
[['NONE', '2600', 'C', datetime.datetime(2020, 9, 16, 0, 0)]]
[['VXX', '45', 'c', datetime.datetime(2020, 7, 10, 0, 0)], ['XLF', '22', 'p', datetime.datetime(2020, 6, 19, 0, 0)]]
[['SPY', '230', 'p', datetime.datetime(2020, 9, 19, 0, 0)], ['SPY', '200', 'p', datetime.datetime(2020, 12, 19, 0, 0)]]
[['EA', '145', 'c', datetime.datetime(2020, 7, 2, 0, 0)], ['ATVI', '78', 'c', datetime.datetime(2020, 6, 26, 0, 0)], ['SNE', '75', 'c', datetime.datetime(2020, 6, 19, 0, 0)], ['ATVI', '78', 'c', datetime.datetime(2020, 7, 2, 0, 0)], ['SNE', '75', 'c', datetime.datetime(2020, 6, 26, 0, 0)]]
[['ITB', '44.5', 'C', datetime.datetime(2020, 6, 19, 0, 0)], ['LEN', '65', 'C', datetime.datetime(2020, 6, 19, 0, 0)]]
[['NKLA', '100', 'c', datetime.datetime(2020, 6, 19, 0, 0)]]
[['INTC', '65', 'c', datetime.datetime(2020, 7, 2, 0, 0)], ['INTC', '64.5', 'c', datetime.datetime(2020, 7, 2,

[['MSFT', '210', 'C', datetime.datetime(2020, 7, 31, 0, 0)]]
[['MY', '40', 'C', datetime.datetime(2020, 8, 21, 0, 0)], ['MY', '40', 'C', datetime.datetime(2020, 8, 21, 0, 0)], ['MY', '36', 'C', datetime.datetime(2020, 9, 18, 0, 0)]]
[['NONE', '6', 'c', datetime.datetime(2020, 10, 16, 0, 0)]]
[['ACI', '20', 'C', datetime.datetime(2020, 8, 21, 0, 0)]]
[['SSTK', '42.50', 'c', datetime.datetime(2020, 11, 20, 0, 0)]]
[['AMD', '67.5', 'C', datetime.datetime(2020, 8, 21, 0, 0)]]
[['SPY', '324', 'p', datetime.datetime(2020, 7, 24, 0, 0)]]
[['TWTR', '39', 'P', datetime.datetime(2020, 7, 24, 0, 0)], ['TWTR', '37', 'P', datetime.datetime(2020, 7, 31, 0, 0)]]
[['NONE', '80', 'p', datetime.datetime(2020, 8, 24, 0, 0)]]
[['$CLX', '230', 'C', datetime.datetime(2020, 8, 28, 0, 0)]]
[['NONE', '5', 'c', datetime.datetime(2020, 7, 31, 0, 0)]]
[['NONE', '66', 'p', datetime.datetime(2020, 7, 31, 0, 0)]]
[['MDLZ', '55', 'c', datetime.datetime(2020, 7, 31, 0, 0)], ['MDLZ', '60', 'c', datetime.datetime(2020, 

[['SLV', '28.5', 'c', datetime.datetime(2020, 8, 21, 0, 0)]]
[['NONE', '30', 'c', datetime.datetime(2020, 10, 16, 0, 0)]]
[['NONE', '500', 'C', datetime.datetime(2020, 9, 4, 0, 0)]]
[['Q', '38', 'c', datetime.datetime(2020, 11, 20, 0, 0)], ['Q', '38', 'c', datetime.datetime(2021, 3, 15, 0, 0)]]
[['AAPL', '500', 'c', datetime.datetime(2020, 9, 4, 0, 0)]]
[['TSLA', '3500', 'C', datetime.datetime(2020, 11, 20, 0, 0)]]
[['AMD', '90', 'C', datetime.datetime(2020, 9, 4, 0, 0)]]
[['NONE', '700', 'c', datetime.datetime(2020, 9, 18, 0, 0)]]
[['AAPL', '550', 'c', datetime.datetime(2020, 9, 4, 0, 0)]]
[['$RST', '30.00', 'C', datetime.datetime(2020, 9, 18, 0, 0)]]
[['NONE', '500', 'C', datetime.datetime(2020, 8, 28, 0, 0)], ['NONE', '490', 'C', datetime.datetime(2020, 8, 28, 0, 0)], ['C', '490', 'C', datetime.datetime(2020, 8, 28, 0, 0)]]
[['NONE', '80', 'c', datetime.datetime(2020, 9, 18, 0, 0)]]
[['NONE', '90', 'p', datetime.datetime(2020, 8, 28, 0, 0)]]
[['RKT', '30', 'c', datetime.datetime(202

[['NONE', '50', 'c', datetime.datetime(2020, 2, 21, 0, 0)]]
[['NONE', '100', 'P', datetime.datetime(2020, 10, 16, 0, 0)]]
[['NONE', '82', 'c', datetime.datetime(2020, 11, 20, 0, 0)]]
[['SQ', '190', 'c', datetime.datetime(2020, 10, 23, 0, 0)]]
[['NONE', '3', 'c', datetime.datetime(2021, 1, 15, 0, 0)]]
[['$NERD', '26', 'c', datetime.datetime(2020, 10, 16, 0, 0)], ['$NERD', '29', 'c', datetime.datetime(2021, 1, 15, 0, 0)], ['$ESPO', '64', 'c', datetime.datetime(2020, 10, 16, 0, 0)], ['$ESPO', '69', 'c', datetime.datetime(2021, 1, 15, 0, 0)]]
[['AAPL', '123', 'C', datetime.datetime(2020, 10, 30, 0, 0)]]
[['NONE', '700', 'c', datetime.datetime(2020, 12, 22, 0, 0)], ['NONE', '600', 'c', datetime.datetime(2020, 12, 22, 0, 0)]]
[['NONE', '27.5', 'P', datetime.datetime(2021, 4, 16, 0, 0)]]
[['NONE', '150', 'c', datetime.datetime(2021, 1, 15, 0, 0)]]
[['ENPH', '110', 'P', datetime.datetime(2020, 11, 20, 0, 0)]]
[['TSLA', '460', 'C', datetime.datetime(2020, 10, 30, 0, 0)], ['TSLA', '600', 'C', da

In [ ]:
## work on finding the missing tickers

In [24]:
x = good_post_csv.apply(ticker_finder, axis=1)

T
HLT
AMZN
HDB
BLK
GE
A
STZ
STZ
ATH
V
SPCE
TEVA
TEVA
TEVA
TEVA
ON
SPCE
SEAS
CCL
CCL
SEAS
BBBY
Z
MGM
GE
F
CCL
FLIR
FLIR
FLIR
FLIR
FLIR
FLIR
PE
PE
PE
FLIR
FLIR
FLIR
WYNN
DD
WYNN
YUMC
LVS
BAC
SPCE
SQ
AMC
SQ
SQ
SQ
SQ
WING
DD
HSY
DE
HSY
BP
ATH
ATH
PLAY
PLAY
PLAY
PLAY
A
EOD
PLAY
EOD
T
USA
LOW
LOW
SPCE
SPCE
PLNT
PLNT
PLNT
A
CEO
NYC
AAPL
AAPL
TWTR
GE
NKE
LULU
NKE
WELL
SPCE
GE
GE
MTCH
MAR
TEVA
SNAP
GPRO
OUT
TSLA
TSLA
INSG
XOM
AMD
ZNGA
ZNGA
ZNGA
ZNGA
PDT
AMD
TSLA
TSLA
SPCE
NIO
CRSP
BA
SPCE
FB
BMO
DPZ
DPZ
SPCE
ATH
NOW
SPCE
KR
KR
NVDA
PLUG
TSM
AMD
NVDA
AAPL
IBM
TSM
AMD
IBM
EPAM
ZG
ARE
ENPH
ENPH
NVDA
LB
L
CEO
A
L
L
NOK
ALLY
ALLY
SPWR
NEW
SPCE
LL
ALLY
ALLY
FIT
DBX
A
RH
RH
RH
NOMD
RH
RH
A
HPE
HPE
HPQ
HPE
A
BA
BA
BA
BA
VIRT
COST
TD
DIS
A
DIS
SPCE
SPCE
SPCE
A
AAL
DAL
CDC
USA
USA
USA
USA
C
NYC
MSFT
AMZN
AMZN
MSFT
MSFT
AMZN
TGT
ARMK
DD
ARMK
CTXS
GILD
ABT
MRNA
CODX
INO
PS
PLAY
PLAY
A
BA
KKR
A
A
A
PLAY
A
GOOD
ARE
A
ARE
BY
FOR
A
GOOD
A
OR
ARE
A
IT
BE
A
A
ONE
A
FOR
A
PLAY
SEAS
F
PLAY
PLAY
PLAY
PLAY
PLAY
PE
T

SFIX
ZM
DD
DX
NKLA
NKLA
GM
NKLA
TSLA
NVTA
ON
DD
DD
TSLA
TLT
PZZA
TSLA
TSLA
VALE
TSLA
MSFT
AMZN
GOOG
MSFT
VALE
VALE
SNAP
API
NKLA
NKLA
TSLA
SPH
A
SPH
SPH
SPH
A
SPH
SPH
GME
OI
TSLA
RKT
RKT
NIO
NIO
SMH
SOXX
SMH
SOXX
SMH
SOXX
AAPL
AAPL
IRBT
IRBT
QSR
QSR
AMD
DD
DD
X
X
SO
MSFT
AMZN
MSFT
MSFT
MSFT
MSFT
MSFT
MSFT
MSFT
SYK
SYK
XLNX
MRNA
CHTR
AMD
AMD
TSLA
TSLA
TSLA
SPCE
GE
GE
CEO
GE
MMM
CVX
XOM
IBM
DOW
PFE
WBA
KO
CSCO
VZ
DD
WBA
TA
DTE
MMM
VZ
DTE
CVX
DTE
CSCO
DTE
PFE
DTE
AMD
PCG
RH
PCG
PE
PTON
SDC
DD
TSLA
TSLA
CSCO
CSCO
DIS
LLY
JPM
PTON
GRUB
SHAK
ACB
DD
T
X
Y
Z
T
CRM
DFS
MSFT
SNAP
NOK
NOK
MAXR
DFS
MSFT
SNAP
MSFT
MSFT
MSFT
MRNA
INO
VIR
MRNA
JNJ
JNJ
JNJ
UBER
MSFT
MSFT
PCG
PCG
PCG
FB
FB
MSFT
MSFT
REAL
JNPR
MSFT
SNAP
KHC
PDD
UBER
UBER
PCG
LULU
AMD
CHGG
CRSP
CRSP
QQQ
DOW
AAPL
MSFT
TSLA
AMD
GOOG
ROKU
ROKU
ROKU
NYC
HAS
MSFT
HAS
MSFT
CVS
MSFT
SPCE
MSFT
SPCE
MSFT
SPCE
MSFT
SPCE
TSLA
ARE
AT
RH
RH
ALL
ARE
A
A
IT
IT
OR
AT
ST
SMH
STNE
MSFT
STNE
PHO
MSFT
MSFT
SPCE
NOK
SPCE
SPCE
SPCE
X
SPCE
SPCE
SPCE
BABA
CSCO


NOW
NOW
A
O
A
GO
GO
O
DD
CEO
UNH
UNH
UNH
OLLI
TTD
YELP
RVLV
DD
RVLV
YELP
BBBY
BBBY
BBBY
BBBY
PE
BBBY
QQQ
A
FOR
DIS
SEAS
BLMN
GM
DRI
YUM
BA
GME
BBBY
DD
QQQ
TA
A
OR
B
D
SQQQ
ULTA
MSFT
DD
BOOM
WWE
JPM
ULTA
BAC
F
F
MGM
MGM
F
PK
F
DD
DD
DOW
SHOO
UBER
DD
AIG
AIG
BAC
AIG
GES
WYNN
MGM
LVS
CZR
AAL
BAC
WHR
OUT
SIRI
SHO
BBY
GE
GPS
CRI
DIS
QQQ
PTON
TSLA
SHOP
FB
WWE
SPCE
SBUX
ARMK
T
LYV
SEAS
SIX
UBER
SLM
DB
LYFT
BYND
CMG
WING
TXRH
BLMN
OHI
OHI
OHI
A
DD
DD
ATVI
ATVI
BR
DD
HSY
DD
MSFT
A
SMFG
EOD
WDAY
DIS
ATVI
EOD
GE
GPRO
F
BAC
GM
DLTR
USA
SYY
SYY
SYY
SYY
IBKR
YELP
DIS
GM
DD
TSLA
TSLA
OI
OI
UBER
LYFT
PINS
F
OI
OI
OI
UBER
LYFT
PINS
F
F
F
VERY
MGM
Z
GME
UBER
DD
PSEC
PSEC
CMO
VCSH
VCIT
PSEC
PSEC
PSEC
EOD
PTON
DD
PTON
PTON
DD
TDOC
TDOC
TDOC
DIS
DIS
DG
PE
DG
ROST
DD
KEY
CCRN
AA
CZR
FOR
CZR
ROKU
RH
RCL
LULU
LYFT
MR
TR
THR
SP
SP
A
CME
A
SNAP
CHWY
NYC
GILD
C
SBUX
DD
TCOM
HCA
HCA
TNET
MTCH
DD
IAC
PE
MTCH
DD
PTON
ON
OR
WORK
MS
WORK
HUM
CAH
ANTM
TDOC
HUM
CAH
ANTM
TDOC
DD
SBUX
DT
USA
ALL
SBUX
GME
HOG
EV
CEO
CEO
C

A
Z
A
DD
Z
Z
CEO
CEO
CEO
Z
MSFT
MSFT
AMZN
TSM
CFO
TSM
DD
TSM
NOK
AAPL
NOK
DD
GILD
ABT
ABT
DD
DD
TAK
TWO
TAK
TAK
CRWD
TENB
PFPT
DD
DD
TAK
TWO
TAK
TAK
TAK
HROW
HROW
TSLA
ABT
UNH
GILD
B
FOR
AN
USA
USA
AMZN
ET
NOK
OXY
TA
TA
FUND
PS
QQQ
SBUX
LTRPA
DOW
TA
BA
SPCE
TSLA
MSFT
GSK
GSK
OI
OI
TAP
STZ
DEO
APA
OKE
PLOW
BY
TWO
WTI
APA
OKE
APA
OKE
B
C
PB
A
B
C
ABC
A
A
FLWS
FLWS
BA
BA
BA
BA
ACB
GER
SE
RH
DD
NYC
FICO
DD
A
A
HAL
MSFT
MSFT
MSFT
MSFT
GILD
GM
DD
GM
GM
GM
H
ZM
ZM
ZM
DIS
BA
AMZN
GSX
GSX
GSX
GSX
DIS
NFLX
NFLX
NFLX
NFLX
ALL
DIS
FOX
DIS
DIS
GILD
LYV
REAL
DIS
DIS
UBS
DIS
SHOP
APRN
QQQ
WTI
V
WTI
OXY
EPD
WTI
WTI
Z
CVX
M
A
CVX
PLAY
AT
ARE
A
FVRR
SHOP
AMZN
FVRR
SHOP
FVRR
FVRR
VEEV
A
CRM
CRM
TMO
ACB
DD
FLIR
FLIR
DD
FLIR
FLIR
SO
FLIR
DD
PM
DD
EOD
ES
AMZN
ES
AMZN
TSLA
AMD
SPCE
GILD
MRNA
INO
ALL
MRNA
INO
MRNA
MRNA
MRNA
INO
MRNA
INO
MRNA
MRNA
INO
MRNA
INO
MRNA
INO
HEAR
DD
CNCR
KIDS
A
ACB
A
A
ACB
ABBV
AZN
MSFT
C
DAL
FLY
DAL
DAL
PS
TSLA
DIS
GEF
GEF
GEF
GEF
A
BA
IBM
SA
RTX
BA
ABT
IT
DD
A
DB
AIG
A
DB
DB
MA
AT

WIX
WIX
WIX
WIX
WIX
WIX
WIX
WIX
WIX
WIX
RAD
JD
JD
JD
JD
JD
USA
PDD
PDD
JD
JD
PDD
PDD
JD
JD
PDD
PDD
JD
PDD
JD
PDD
JD
JD
JD
JD
JD
JD
JD
PDD
BABA
JD
JD
JD
JD
HP
JD
A
JD
JD
JD
JD
EOD
DG
DG
DG
DG
PE
PE
CEO
DG
TA
DG
DG
DG
CVS
LOGI
SDC
CSCO
TTWO
HD
TGT
BABA
OKTA
LYFT
D
ST
ST
ST
CEO
EXPR
PS
ALL
A
OI
CSCO
OI
CSCO
OI
DD
DD
A
DD
DD
OKTA
CDLX
SDC
DD
CSCO
LOGI
DD
BOX
SATS
SATS
DD
OKTA
CDLX
DD
DD
PLAY
AT
EB
AMC
FREE
DD
EB
EB
TLRY
DD
EVRI
EVRI
EVRI
EVRI
EVRI
EVRI
EVRI
EVRI
EVRI
TGT
TGT
DD
ATH
SDC
SDC
SDC
HTZ
GOOD
CEO
CVS
HOG
GT
HOG
A
BIG
BIG
TGT
CSCO
NET
DD
EVRI
DD
ET
EVRI
EVRI
CZR
EVRI
EVRI
EVRI
EVRI
EVRI
EVRI
EVRI
EVRI
DD
WYNN
CRM
DBX
KR
AMD
AMD
WYNN
A
CRM
CRWD
DDOG
ESTC
NOW
OKTA
TEAM
TEAM
WDAY
WORK
CEO
ZS
ZS
BYD
EVRI
TRWH
WYNN
NVAX
ALT
VXRT
HTBX
BNTX
TTWO
TTWO
DD
DD
DD
TKR
DD
GO
DD
EGHT
EGHT
DD
EGHT
EGHT
EGHT
EGHT
MC
MC
C
D
EV
CD
RH
CSV
DD
DD
III
CRM
CRM
CLX
RNG
TECH
DD
TECH
HD
RH
RH
DG
DG
DG
DG
DG
PE
PE
CEO
DG
TA
DG
DG
DG
CVS
LOGI
HP
HPQ
HP
HPQ
R
BHLB
GWB
MCBS
BCOW
GWB
BHLB
BCOW
MCBS
UNB
NKSH
NCB

...

TV
AVID
TV
TV
DD
BLNK
BLNK
TECH
PLNT
SIX
HLT
SMPL
SMPL
SMPL
SMPL
SMPL
SMPL
SMPL
SMPL
SMPL
PRPL
PRPL
PRPL
PRPLW
PRPL
DD
A
DD
CASH
PRPL
PRPL
PE
TPX
SNBR
PRPL
EV
PRPL
EV
CSPR
CSPR
PRPL
CSPR
CSPR
A
CSPR
PRPL
PRPL
PRPL
TPX
SNBR
PRPL
EV
R
PRPL
SNBR
EV
R
EV
R
PRPL
CEO
PRPL
CSPR
PRPL
PRPL
PRPL
PRPL
PRPLW
PRPL
PRPL
CSPR
PRPL
CSPR
PRPL
CSPR
PRPL
PRPL
CSPR
CSPR
PRPL
CSPR
PRPL
FB
AMC
AMC
AMC
AMC
AMC
NVAX
DD
DD
NVAX
NVAX
INO
INO
NVAX
DD
RNA
IP
INO
NVAX
INO
DD
DD
TSCO
TSCO
TSCO
TSCO
A
PE
DD
DD
TIF
AGM
MD
SA
SA
MSFT
PTON
LULU
PTON
LULU
PTON
LULU
LULU
HTZ
EOD
R
PCG
A
BK
BK
CDC
A
A
MSFT
NET
SE
ITCI
ITCI
DIS
ODP
ODP
ODP
JBHT
FDX
UPS
FDX
JBHT
FDX
UPS
JBHT
DD
PLNT
PLNT
PLNT
PLNT
PLNT
PLNT
PLNT
PTON
PLNT
LULU
NYC
ONE
PLNT
PLNT
PLNT
PLNT
PLNT
PLNT
PLNT
PLNT
PLNT
PLNT
FSLY
PLNT
FSLY
ZM
FB
LMT
LMT
RIG
GSX
GSX
GSX
GSX
GSX
GSX
GSX
GSX
GSX
GSX
GSX
EBAY
EBAY
TLC
EBAY
EBAY
GDDY
PS
AMZN
AAPL
MSFT
GOOG
NFLX
MSFT
NIO
TDA
TDA
NIO
A
A
PEP
PEP
PEP
PEP
SCHW
SCHW
TD
SCHW
TD
DD
SCHW
SPWR
IEA
SPWR
SPWR
SPWR
SPWR
PFE
PFE
ERI

WMT
DD
WMT
WMT
BUD
BUD
BOTZ
BOTZ
SO
FB
DB
FB
FB
FB
FB
FB
FB
AMD
OPK
SBUX
MU
AAPL
AAPL
FB
BLKB
DD
A
A
BBC
NVDA
NET
MSFT
KODK
DD
DD
EOD
MSFT
TSLA
AMZN
AMZN
FB
MSFT
MSFT
MSFT
FB
AMZN
MSFT
FB
MSFT
SHOP
EV
SHOP
A
BA
BA
BA
PRPL
PE
PRPL
C
GLW
A
GLW
GLW
GLW
CVS
CVS
CVS
CVS
CVS
FB
FB
AA
ELY
ELY
PYPL
PYPL
PLAY
ATH
ATH
TSLA
TSLA
TSLA
GOLD
GOLD
SQ
SQ
SQ
GM
EV
FORD
FORD
EV
ATH
FB
BC
CEO
CEO
WORK
A
CEO
CEO
CEO
A
A
CEO
WORK
GILD
C
ADMP
ADMP
ADMP
PRPL
DD
CEO
PRPL
PRPL
FLIR
DD
FLIR
FLIR
FLIR
DD
FLIR
PLUS
FLIR
FLIR
GPN
UPS
GPN
GPN
SQ
DD
AZN
MRK
AZN
MRK
MRNA
INO
NVAX
VXRT
MRNA
PFE
PFE
IP
JPM
PFE
AZN
MRNA
RH
AAPL
UPS
RARE
DD
SF
C
NOK
LOGI
AMD
PE
AMD
LOGI
LOGI
DD
AMD
TSM
TSM
TSM
PCG
PCG
SA
TA
PCG
C
ADMA
DD
DD
ADMA
DD
ADMA
ADMA
ADMA
PS
DD
ANY
B
B
BNGO
KO
SAM
KO
KO
DTE
AM
KO
AAPL
AAPL
GOOGL
AMZN
MSFT
NFLX
AAPL
AAPL
AAPL
AAPL
V
A
V
V
AAPL
V
AAPL
V
AAPL
AAPL
TSLA
UPS
AVYA
JP
AVYA
ABBV
ABBV
DD
DKNG
SQ
SQ
EOD
AAXN
AAXN
AAXN
AAXN
A
ATH
SRNE
TWOU
TWOU
PINS
PINS
PINS
PRPL
UPS
DD
AAL
UAL
AAL
UAL
DD
AAL
KO
KO
A
CAN
K

CVLT
WORK
WORK
WORK
ATH
WORK
WORK
WORK
WORK
MESO
ZEUS
MESO
DBX
IQ
RKT
BCRX
DLTR
DG
DLTR
DG
DLTR
DG
DG
DLTR
DLTR
DG
DLTR
DLTR
PE
DLTR
DG
DLTR
C
HD
LOW
TGT
BBY
LOW
TGT
BBY
WMT
WMT
PE
WMT
WMT
WMT
WMT
WMT
WMT
ARE
A
ATVI
VVNT
VVNT
VVNT
JP
VVNT
CEO
VVNT
VVNT
VVNT
VVNT
RRR
DD
MGM
CEO
III
RKT
PRPL
BA
BA
BA
BA
BA
BA
BA
BA
BA
BA
BA
BA
DIS
DKNG
BOX
DD
DD
DBX
BOX
BOX
BOX
BOX
DBX
XRX
AAPL
AAPL
TSLA
AMZN
NKE
TSLA
SPLK
GB
DDOG
DDOG
AMZN
SPLK
ESTC
DDOG
SPLK
DDOG
ESTC
DDOG
ESTC
ESTC
AMZN
SPLK
SPLK
ESTC
MRK
CEO
DD
TLT
ATH
TLT
TLT
TLT
TLT
TLT
TLT
TLT
TLT
TLT
TLT
HAS
TLT
TLT
TLT
TLT
TLT
HAS
ATH
TLT
TLT
TLT
TLT
TLT
ATH
TLT
TJX
NET
NET
FSLY
NIO
TSLA
FSLY
NET
FSLY
NET
DD
TSLA
MCK
MCK
MCK
CNK
AMC
AMC
AMC
AMC
CEO
MSFT
CNK
MAR
CNK
WMT
DAL
DAL
AAPL
JD
BABA
PTON
PTON
PTON
ATH
DOCU
WOOD
WOOD
WOOD
WOOD
WOOD
WOOD
WOOD
RYN
PCH
WY
LL
WOOD
WOOD
WOOD
WOOD
RYN
PCH
WY
LL
MU
NVDA
RTX
MU
MU
MU
NVDA
MU
A
UPS
XRX
XRX
TSLA
NCNO
NCNO
NCNO
NCNO
NCNO
NCNO
NCNO
NCNO
NCR
ICE
DD
DD
IRR
ICE
ICE
ICE
NVDA
NVDA
NVDA
NVDA
NVDA
NVDA
NVDA


AAPL
AAPL
SQ
SQ
COST
PLUG
DD
WORK
DOW
DOW
FB
MSFT
TSM
TSLA
TSLA
AAPL
AMD
TSLA
TQQQ
PRPL
NVDA
NVDA
TSLA
X
TSLA
TSLA
TSLA
PTON
PTON
AAPL
AAPL
TSLA
WMT
TSLA
EPD
TSLA
AAPL
ZM
FDX
FDX
MSFT
SO
NOW
AAPL
AAL
TSLA
AAPL
AAPL
AAPL
AAPL
TSLA
CDC
AAPL
AAPL
TSLA
AAPL
AMZN
FB
JPM
GS
BAC
AAL
UAL
JPM
BAC
UAL
TD
AMD
AAPL
RH
RH
LOGI
INTC
MA
TSLA
EVER
TSLA
A
FDX
DD
TA
ATH
WING
WING
ATH
WING
GO
MSFT
MSFT
TSLA
DD
PRPL
AMD
AAPL
DD
AAPL
SABR
PRPL
RKT
DD
PRPL
A
OI
OI
SNE
SNE
PS
CHWY
CRM
CSCO
CHWY
CHWY
TSLA
TSLA
EV
TSLA
TSLA
TSLA
TSLA
TSLA
TSLA
SNE
AAPL
MSFT
NVDA
SNE
J
TSLA
TSLA
TSLA
PINS
BIGC
FB
PINS
QCOM
AVGO
TXN
QCOM
QCOM
TA
QCOM
QCOM
AMD
AAPL
ETSY
AAL
C
QQQ
C
AAPL
C
C
INTC
C
PS
A
RH
NVDA
PS
NIO
NIO
LOW
PE
NVDA
TQQQ
TSLA
ADBE
ADBE
C
ZS
MMM
ZM
ZS
ZS
ZM
ZS
MSFT
AAPL
WMT
CHWY
PTON
FB
A
TSLA
FB
TSLA
TSLA
TSLA
QCOM
AAPL
QCOM
AAPL
QCOM
QCOM
DD
DD
FDX
FDX
FDX
TSLA
TSLA
AAPL
USA
KMX
KMX
USA
KMX
KMX
KO
NCLH
NCLH
NCLH
NCLH
SAVE
SAVE
TSM
MS
DIS
CHWY
GIS
PTON
PTON
SLQT
HP
AAPL
AAPL
AAPL
TSLA
AAPL
DD
AAPL
AAPL
AAPL
AMD
A

SPCE
DD
PG
CDC
A
TSLA
PG
NIO
ZM
ZM
ZM
ONE
BABA
BABA
BABA
BABA
ES
BJ
COST
DD
DD
BJ
OXY
OXY
OXY
OXY
OXY
ALB
DD
ALB
NKLA
NKLA
GM
GM
NKLA
NKLA
NKLA
NKLA
RGR
RGR
RGR
A
C
GM
ASML
BABA
BABA
BABA
RBC
BABA
BABA
BABA
BABA
BABA
BABA
A
EDIT
KSS
KSS
TSLA
DD
OI
EOD
OI
SNAP
OI
STM
OI
WMT
OI
NIO
NIO
NIO
ICLN
RUN
C
DD
TTCF
PACB
PACB
NKLA
RH
PLUG
GM
SPCE
SPCE
SPCE
MGM
CHWY
CHWY
CHWY
SHOP
SHOP
QQQ
SHOP
SHOP
SYRS
SYRS
SYRS
J
MSFT
MSFT
GME
NKLA
TSLA
TSLA
GM
GM
GM
NEXT
VIPS
TA
VIPS
PT
VIPS
BABA
BABA
BABA
VIPS
AMD
SBUX
A
SBUX
SBUX
SBUX
CEO
SBUX
AMD
AMD
AMD
AMD
AMD
A
TSLA
HSBC
TSLA
SF
GS
JPM
JPM
GS
JPM
SDC
GILD
WMT
AMZN
WMT
WMT
WMT
WMT
AMZN
AMZN
WMT
JLL
DD
WMT
WMT
WMT
OZK
OZK
NYC
OZK
OZK
OZK
BAC
DD
CNC
ATVI
ATVI
ICLN
L
VALE
VALE
CNK
DD
CNK
NYC
CFO
A
AMC
AAPL
ICLN
AMD
AMD
BJ
BJ
BJ
BIG
BJ
A
BJ
BJ
DD
BJ
ESG
PE
JKS
CSIQ
JKS
UPS
UPS
UPS
UPS
UPS
UPS
UPS
JBHT
AMD
AMD
AMD
AMD
AMD
ICLN
ICLN
XSPA
NKLA
UPS
UPS
UPS
UPS
SPCE
SPCE
SPCE
NKLA
SPCE
SPCE
SPCE
SPCE
SCI
SCI
SO
SCI
SCI
USA
SCI
SCI
PLAY
NKLA
BJ
BJ
BIDU
BIDU
AMD
UP

In [25]:
for i,v in enumerate(x):
    print(i, v)

0 None
1 None
2 None
3 None
4 None
5 None
6 Counter({'T': 1})
7 None
8 None
9 None
10 None
11 None
12 None
13 None
14 None
15 None
16 None
17 None
18 None
19 None
20 None
21 None
22 None
23 None
24 None
25 None
26 None
27 None
28 None
29 None
30 None
31 None
32 None
33 None
34 None
35 None
36 None
37 None
38 None
39 None
40 None
41 None
42 None
43 None
44 None
45 None
46 None
47 None
48 None
49 None
50 None
51 None
52 None
53 None
54 None
55 None
56 None
57 None
58 None
59 None
60 None
61 None
62 None
63 None
64 None
65 None
66 None
67 None
68 None
69 None
70 None
71 None
72 None
73 None
74 None
75 None
76 None
77 None
78 None
79 None
80 None
81 None
82 None
83 None
84 None
85 None
86 None
87 None
88 None
89 None
90 None
91 None
92 None
93 None
94 None
95 None
96 None
97 None
98 None
99 None
100 None
101 None
102 None
103 None
104 None
105 None
106 None
107 None
108 None
109 None
110 None
111 None
112 None
113 None
114 None
115 None
116 None
117 None
118 None
119 None
120 None
121 None

1036 None
1037 None
1038 None
1039 None
1040 None
1041 None
1042 Counter({'XOM': 1})
1043 None
1044 None
1045 None
1046 None
1047 None
1048 None
1049 None
1050 None
1051 None
1052 None
1053 None
1054 None
1055 None
1056 None
1057 None
1058 None
1059 None
1060 Counter({'AMD': 1})
1061 None
1062 None
1063 None
1064 None
1065 Counter({'ZNGA': 2})
1066 None
1067 Counter({'ZNGA': 2})
1068 None
1069 None
1070 None
1071 None
1072 None
1073 None
1074 None
1075 None
1076 None
1077 None
1078 Counter({'PDT': 1})
1079 None
1080 None
1081 None
1082 None
1083 None
1084 None
1085 None
1086 None
1087 None
1088 None
1089 None
1090 None
1091 Counter({'AMD': 1})
1092 None
1093 None
1094 None
1095 None
1096 None
1097 None
1098 None
1099 None
1100 None
1101 None
1102 None
1103 None
1104 None
1105 None
1106 None
1107 None
1108 None
1109 None
1110 None
1111 None
1112 None
1113 None
1114 None
1115 None
1116 None
1117 None
1118 None
1119 None
1120 None
1121 None
1122 None
1123 None
1124 None
1125 None
1126 Non

2155 None
2156 Counter({'DBX': 1})
2157 None
2158 None
2159 None
2160 None
2161 None
2162 None
2163 None
2164 None
2165 None
2166 None
2167 None
2168 None
2169 None
2170 None
2171 None
2172 None
2173 None
2174 None
2175 None
2176 None
2177 None
2178 None
2179 None
2180 None
2181 None
2182 Counter({'RH': 3, 'A': 1})
2183 None
2184 None
2185 None
2186 None
2187 None
2188 None
2189 None
2190 None
2191 None
2192 None
2193 None
2194 None
2195 None
2196 None
2197 None
2198 None
2199 None
2200 None
2201 None
2202 None
2203 None
2204 None
2205 None
2206 None
2207 None
2208 None
2209 None
2210 None
2211 None
2212 None
2213 None
2214 None
2215 None
2216 None
2217 Counter({'RH': 2, 'NOMD': 1})
2218 None
2219 None
2220 None
2221 None
2222 None
2223 None
2224 None
2225 None
2226 None
2227 None
2228 None
2229 None
2230 None
2231 None
2232 None
2233 None
2234 None
2235 None
2236 None
2237 None
2238 None
2239 None
2240 None
2241 None
2242 None
2243 None
2244 None
2245 None
2246 None
2247 None
2248 Non

3255 None
3256 None
3257 None
3258 None
3259 None
3260 None
3261 None
3262 None
3263 None
3264 None
3265 None
3266 None
3267 None
3268 None
3269 None
3270 None
3271 None
3272 None
3273 None
3274 None
3275 None
3276 None
3277 None
3278 None
3279 None
3280 None
3281 None
3282 Counter({'OSTK': 4, 'DD': 1})
3283 None
3284 None
3285 None
3286 Counter({'PM': 2})
3287 None
3288 None
3289 None
3290 None
3291 None
3292 None
3293 None
3294 Counter({'NOW': 1, 'EDIT': 1, 'SQQQ': 1})
3295 Counter()
3296 None
3297 None
3298 None
3299 None
3300 None
3301 None
3302 None
3303 None
3304 None
3305 None
3306 None
3307 None
3308 None
3309 None
3310 None
3311 None
3312 None
3313 None
3314 None
3315 None
3316 None
3317 None
3318 None
3319 None
3320 None
3321 None
3322 None
3323 Counter({'DD': 1})
3324 None
3325 None
3326 None
3327 None
3328 None
3329 None
3330 None
3331 None
3332 None
3333 None
3334 None
3335 None
3336 None
3337 None
3338 None
3339 None
3340 None
3341 None
3342 None
3343 None
3344 None
3345 

4369 Counter({'PTON': 3, 'DD': 1})
4370 None
4371 None
4372 None
4373 None
4374 None
4375 None
4376 None
4377 None
4378 None
4379 None
4380 None
4381 None
4382 None
4383 None
4384 None
4385 None
4386 None
4387 None
4388 None
4389 Counter()
4390 None
4391 None
4392 None
4393 None
4394 None
4395 None
4396 Counter({'TSLA': 1})
4397 Counter({'A': 1, 'V': 1})
4398 Counter()
4399 None
4400 None
4401 None
4402 None
4403 None
4404 None
4405 None
4406 None
4407 None
4408 None
4409 None
4410 None
4411 None
4412 None
4413 None
4414 None
4415 None
4416 None
4417 None
4418 None
4419 None
4420 None
4421 None
4422 None
4423 None
4424 None
4425 Counter({'GE': 5})
4426 None
4427 None
4428 None
4429 None
4430 None
4431 None
4432 None
4433 None
4434 None
4435 None
4436 None
4437 None
4438 None
4439 None
4440 None
4441 None
4442 None
4443 Counter({'FUND': 1})
4444 None
4445 None
4446 None
4447 None
4448 None
4449 None
4450 None
4451 None
4452 None
4453 None
4454 None
4455 None
4456 None
4457 None
4458 Non

...

112833 None
112834 None
112835 None
112836 None
112837 None
112838 None
112839 Counter({'GM': 1})
112840 None
112841 None
112842 None
112843 None
112844 None
112845 Counter({'CGC': 2})
112846 None
112847 None
112848 None
112849 None
112850 None
112851 None
112852 Counter()
112853 None
112854 None
112855 None
112856 None
112857 None
112858 None
112859 None
112860 None
112861 None
112862 None
112863 None
112864 None
112865 None
112866 None
112867 None
112868 None
112869 None
112870 None
112871 Counter({'CROX': 2})
112872 None
112873 None
112874 None
112875 None
112876 None
112877 None
112878 Counter()
112879 None
112880 None
112881 None
112882 Counter({'FSLR': 4})
112883 None
112884 None
112885 None
112886 None
112887 None
112888 None
112889 None
112890 None
112891 Counter({'IMAX': 4})
112892 None
112893 None
112894 None
112895 None
112896 None
112897 None
112898 None
112899 None
112900 None
112901 None
112902 None
112903 None
112904 None
112905 None
112906 None
112907 None
112908 None
1

113940 None
113941 None
113942 None
113943 None
113944 None
113945 None
113946 None
113947 None
113948 None
113949 None
113950 None
113951 None
113952 None
113953 None
113954 None
113955 None
113956 None
113957 None
113958 None
113959 None
113960 None
113961 None
113962 None
113963 None
113964 None
113965 None
113966 None
113967 None
113968 None
113969 None
113970 None
113971 None
113972 None
113973 None
113974 None
113975 Counter({'FVAC': 5, 'EV': 4, 'DD': 1, 'SPCE': 1, 'PLTR': 1, 'WORK': 1, 'BOX': 1, 'A': 1, 'CEO': 1, 'NKLA': 1, 'TSLA': 1})
113976 None
113977 None
113978 None
113979 None
113980 None
113981 None
113982 None
113983 None
113984 None
113985 None
113986 None
113987 Counter()
113988 None
113989 None
113990 None
113991 None
113992 None
113993 None
113994 None
113995 Counter({'F': 5})
113996 None
113997 None
113998 None
113999 None
114000 None
114001 None
114002 None
114003 None
114004 None
114005 None
114006 None
114007 None
114008 None
114009 None
114010 Counter({'SNAP': 1

115065 None
115066 Counter({'NOK': 1})
115067 None
115068 None
115069 Counter({'FB': 2})
115070 None
115071 None
115072 None
115073 None
115074 None
115075 None
115076 Counter({'NIO': 2, 'PLAY': 1})
115077 None
115078 None
115079 Counter()
115080 None
115081 None
115082 None
115083 None
115084 None
115085 None
115086 None
115087 None
115088 None
115089 None
115090 None
115091 None
115092 None
115093 None
115094 None
115095 None
115096 None
115097 None
115098 None
115099 None
115100 Counter()
115101 None
115102 None
115103 None
115104 None
115105 None
115106 None
115107 None
115108 None
115109 Counter()
115110 Counter()
115111 None
115112 None
115113 None
115114 Counter()
115115 None
115116 None
115117 None
115118 None
115119 None
115120 None
115121 None
115122 None
115123 None
115124 None
115125 None
115126 None
115127 None
115128 None
115129 None
115130 None
115131 None
115132 None
115133 None
115134 None
115135 None
115136 None
115137 None
115138 None
115139 None
115140 None
115141 N

116217 None
116218 None
116219 None
116220 None
116221 None
116222 None
116223 None
116224 None
116225 None
116226 None
116227 None
116228 None
116229 None
116230 None
116231 None
116232 None
116233 None
116234 None
116235 None
116236 None
116237 None
116238 None
116239 None
116240 Counter({'ADT': 6, 'GOOG': 1})
116241 None
116242 None
116243 None
116244 None
116245 None
116246 None
116247 None
116248 None
116249 None
116250 None
116251 None
116252 None
116253 None
116254 None
116255 None
116256 None
116257 None
116258 None
116259 None
116260 None
116261 None
116262 None
116263 None
116264 None
116265 None
116266 None
116267 None
116268 None
116269 None
116270 None
116271 None
116272 None
116273 None
116274 None
116275 None
116276 None
116277 None
116278 None
116279 None
116280 None
116281 None
116282 None
116283 None
116284 None
116285 None
116286 None
116287 None
116288 None
116289 None
116290 None
116291 None
116292 None
116293 None
116294 None
116295 None
116296 None
116297 None
11

117313 None
117314 None
117315 None
117316 None
117317 None
117318 None
117319 None
117320 None
117321 None
117322 None
117323 None
117324 None
117325 None
117326 None
117327 None
117328 None
117329 None
117330 None
117331 None
117332 None
117333 None
117334 None
117335 None
117336 None
117337 None
117338 None
117339 Counter({'KCAC': 2, 'EV': 1})
117340 None
117341 Counter({'RKT': 2, 'SHIP': 2})
117342 None
117343 None
117344 None
117345 None
117346 None
117347 None
117348 None
117349 None
117350 None
117351 None
117352 None
117353 None
117354 None
117355 None
117356 None
117357 None
117358 None
117359 None
117360 None
117361 None
117362 None
117363 None
117364 None
117365 None
117366 None
117367 None
117368 None
117369 None
117370 None
117371 None
117372 None
117373 None
117374 None
117375 None
117376 None
117377 None
117378 None
117379 None
117380 None
117381 None
117382 None
117383 None
117384 None
117385 None
117386 None
117387 None
117388 None
117389 None
117390 None
117391 None
1

In [26]:
good_post_csv.iloc[5741]

Unnamed: 0                                                       5741
author                                                     oohcharlie
created_utc                                                1590065171
title               John ($DE)ere and your chance to harvest some ...
link_flair_text                                                    DD
selftext            Whatsup city slickers? Look out your window. W...
id                                                             go1hzc
full_link           https://www.reddit.com/r/wallstreetbets/commen...
regexed_body                               [[NONE, 75, p, 1/15/2021]]
regexed_title                                                       0
regexed_combined                           [[NONE, 75, p, 1/15/2021]]
Name: 5741, dtype: object

In [27]:
good_post_csv.iloc[5751]

Unnamed: 0                                                       5751
author                                                    epicoliver3
created_utc                                                1590066201
title               EURN (Tanker) should be priced higher, and has...
link_flair_text                                                    DD
selftext            (Plays at end)\n\nThe tanker stock EURN curren...
id                                                             go1tpo
full_link           https://www.reddit.com/r/wallstreetbets/commen...
regexed_body                                    [[NONE, 10, c, 8/13]]
regexed_title                                                       0
regexed_combined                                [[NONE, 10, c, 8/13]]
Name: 5751, dtype: object

In [28]:
good_post_csv

,Unnamed: 0,author,created_utc,title,link_flair_text,selftext,id,full_link,regexed_body,regexed_title,regexed_combined
0,0,i-only-buy-leaps,1569866466,That one where their naked puts got assigned,Options,Some of you degenerates may remember me moanin...,dbjwue,https://www.reddit.com/r/wallstreetbets/commen...,0,0,0
1,1,MJNRJ,1569867181,Atvi 55$ call Drop 5mins before closing can so...,Discussion,So in the last 5 mins today the 55$ call for N...,dbk2w9,https://www.reddit.com/r/wallstreetbets/commen...,0,0,0
2,2,swaggymedia,1569869517,I track YOLOs so you don't have to - September...,Discussion,[removed],dbklrm,https://www.reddit.com/r/wallstreetbets/commen...,0,0,0
3,3,DeepFuckingValue,1569870742,GME YOLO month-end update — Sep 2019,YOLO,NaN,dbkv9r,https://www.reddit.com/r/wallstreetbets/commen...,0,0,0
4,4,KindaOrange,1569871430,NFLX Earnings Post,Discussion,[removed],dbl0r6,https://www.reddit.com/r/wallstreetbets/commen...,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
117648,117648,alexstreerking203,1579692517,Beyondmeat earnings report,Discussion,Can someone please tell me the confirmed date ...,esecee,https://www.reddit.com/r/wallstreetbets/commen...,0,0,0
117649,117649,freshwater21,1579693136,Stock Options Trade Will I recover my losses?,Options,NaN,esehbs,https://www.reddit.com/r/wallstreetbets/commen...,0,0,0
117650,117650,RokasBog,1579693193,"Snapchat has added a bear filter, time for put...",DD,NaN,esehs1,https://www.reddit.com/r/wallstreetbets/commen...,0,0,0
117651,117651,punkwalras,1579693211,Can you guys post your fucking moves?,Discussion,"I'm fucking broke, I got $300 that I wanna put...",esehx2,https://www.reddit.com/r/wallstreetbets/commen...,0,0,0


In [31]:
y = good_post_csv.apply(year_adder, axis = 1)

[['$T', '37.5', 'p', '10/4/2019']]
[['NONE', '200', 'P', '5/15/2020']]
[['HLT', '40', 'P', '5/1/2020']]
[['HDB', '35', 'p', '4/17/2020'], ['EWP', '19', 'p', '4/17/2020'], ['BLK', '295', 'p', '4/17/2020'], ['EEM', '19.73', 'p', '9/18/2020']]
[['VTI', '70', 'p', '4/17/2020']]
[['SPY', '200', 'p', '4/13/2020'], ['SPY', '205', 'p', '4/15/2020'], ['SPY', '200', 'p', '5/1/2020']]
[['V', '167.5', 'c', '3/27/2020']]
[['NONE', '9', 'c', '05/15/2020'], ['NONE', '8.5', 'c', '04/17/2020']]
[['$SPCE', '13.5', 'p', '3/27/2020']]
[['CCL', '13', 'p', '3/27/2020'], ['SEAS', '9', 'p', '3/27/2020']]
[['SPY', '200', 'P', '5/15/2020'], ['BBBY', '3', 'p', '4/17/2020'], ['MGM', '8', 'P', '4/18/2020'], ['MGM', '17.5', 'p', '4/17/2020'], ['INDA', '20', 'p', '4/17/2020'], ['GE', '5', 'p', '4/17/2020'], ['F', '4', 'p', '4/17/2020'], ['CCL', '10', 'p', '4/17/2020']]
[['$FLIR', '35', 'c', '4/17/2020'], ['$FLIR', '40', 'c', '7/17/2020'], ['$FLIR', '50', 'c', '7/17/2020'], ['$FLIR', '35', 'c', '4/17/2020'], ['$FLIR'

ValueError: invalid literal for int() with base 10: '19JUN'